# AWS Data Lake - Udacity STEDI Project

## 1. Import Package

In [4]:
import boto3
import configparser
from time import sleep
import pandas as pd
from botocore.exceptions import ClientError


## 2. Create resource

### Load configuration from aws.cfg

In [5]:
config = configparser.ConfigParser()
config.read('aws.cfg')

AWS_REGION = config.get('AWS', 'AWS_REGION')
AWS_ACCESS_KEY = config.get('AWS', 'AWS_ACCESS_KEY')
AWS_SECRET_ACCESS_KEY = config.get('AWS', 'AWS_SECRET_ACCESS_KEY')
S3_BUCKET_NAME = config.get('S3', 'S3_BUCKET_NAME')
GLUE_DB          = config.get("GLUE","GLUE_DB")

### Create S3 bucket

In [6]:
print("Creating clients for S3")
s3 =  boto3.resource('s3',
                    region_name=AWS_REGION,
                    aws_access_key_id=AWS_ACCESS_KEY,
                    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
                        )

print("Creating AWS S3")
try:
    s3.create_bucket(Bucket=S3_BUCKET_NAME, CreateBucketConfiguration={
        'LocationConstraint': AWS_REGION
    })
except Exception as e:
    if 'BucketAlreadyOwnedByYou' in str(e):
        print(f"Bucket '{S3_BUCKET_NAME}' already exists.")


directories = ['customer',
                'step_trainer', 'accelerometer']
                
for directory in directories:
    bucket = s3.Bucket(S3_BUCKET_NAME)
    s3_key = directory + '/'
    try:
        bucket.Object(s3_key).load()
        print(
            f'S3 directory s3://{S3_BUCKET_NAME}/{s3_key} already exists.')
    except:
        print(f'Creating S3 directory s3://{S3_BUCKET_NAME}/{s3_key}')
        bucket.put_object(Key=s3_key)
        # Wait for the directory to be created
        while True:
            try:
                bucket.Object(s3_key).load()
                print(
                    f'S3 directory s3://{S3_BUCKET_NAME}/{s3_key} created successfully.')
                break
            except:
                print(
                    f'Waiting for S3 directory s3://{S3_BUCKET_NAME}/{s3_key} to be created...')
                sleep(5)

Creating clients for S3
Creating AWS S3


/datadrive05/anhdtv7/miniconda3/envs/aws/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Creating S3 directory s3://anhdtv-stedi/customer/
S3 directory s3://anhdtv-stedi/customer/ created successfully.
Creating S3 directory s3://anhdtv-stedi/step_trainer/
S3 directory s3://anhdtv-stedi/step_trainer/ created successfully.
Creating S3 directory s3://anhdtv-stedi/accelerometer/
S3 directory s3://anhdtv-stedi/accelerometer/ created successfully.


## 3. Sync data from local directory to AWS s3

### Set OS environment variables for AWS CLI

In [7]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
os.environ["AWS_DEFAULT_REGION"] = AWS_REGION

### Sync data from local to S3 bucket

In [72]:
! aws s3 sync ./data/customers/ s3://anhdtv-stedi/customer/landing
# ! aws s3 sync ./data/accelerometer/ s3://anhdtv-stedi/accelerometer/landing
# ! aws s3 sync ./data/step_trainer/ s3://anhdtv-stedi/step_trainer/landing

upload: data/customers/customer-1691348231425.json to s3://anhdtv-stedi/customer/landing/customer-1691348231425.json


### Check data on S3 bucket

In [73]:
s3 = boto3.resource('s3',
                    aws_access_key_id=AWS_ACCESS_KEY,
                    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                    region_name=AWS_REGION  
)

stedi_bucket = s3.Bucket(S3_BUCKET_NAME)

/datadrive05/anhdtv7/miniconda3/envs/aws/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


#### Check Customer data

In [74]:
customer_data = [ obj for obj in stedi_bucket.objects.limit(10).filter(Prefix="customer")]
customer_data

[s3.ObjectSummary(bucket_name='anhdtv-stedi', key='customer/landing/customer-1691348231425.json')]

#### Check Step_trainer data

In [54]:
step_trainer_data = [ obj for obj in stedi_bucket.objects.limit(10).filter(Prefix="step_trainer")]
step_trainer_data

[s3.ObjectSummary(bucket_name='anhdtv-stedi', key='step_trainer/'),
 s3.ObjectSummary(bucket_name='anhdtv-stedi', key='step_trainer/landing/09e31ee8-4138-443a-a985-c538e6433535.csv'),
 s3.ObjectSummary(bucket_name='anhdtv-stedi', key='step_trainer/landing/09e31ee8-4138-443a-a985-c538e6433535.csv.metadata'),
 s3.ObjectSummary(bucket_name='anhdtv-stedi', key='step_trainer/landing/15496593-ba06-4e94-8107-9d357e566c0f.txt'),
 s3.ObjectSummary(bucket_name='anhdtv-stedi', key='step_trainer/landing/step_trainer-1691348232038.json'),
 s3.ObjectSummary(bucket_name='anhdtv-stedi', key='step_trainer/landing/step_trainer-1691348232085.json'),
 s3.ObjectSummary(bucket_name='anhdtv-stedi', key='step_trainer/landing/step_trainer-1691348232132.json'),
 s3.ObjectSummary(bucket_name='anhdtv-stedi', key='step_trainer/trusted/a7dc91a7-78e5-448e-94d8-efb62c373c79.csv'),
 s3.ObjectSummary(bucket_name='anhdtv-stedi', key='step_trainer/trusted/a7dc91a7-78e5-448e-94d8-efb62c373c79.csv.metadata')]

#### Check Accelerometer data

In [12]:
accelerometer_data = [ obj for obj in stedi_bucket.objects.filter(Prefix="accelerometer")]
len(accelerometer_data)

10

## 4. Configuring the S3 VPC Gateway Endpoint

### 4.1 Use the AWS CLI to identify the VPC that needs access to S3:

In [55]:
! aws ec2 describe-vpcs

{
    "Vpcs": [
        {
            "CidrBlock": "172.31.0.0/16",
            "DhcpOptionsId": "dopt-0874ef9faa0280288",
            "State": "available",
            "VpcId": "vpc-0f260bb33946d0513",
            "OwnerId": "004668495805",
            "InstanceTenancy": "default",
            "CidrBlockAssociationSet": [
                {
                    "AssociationId": "vpc-cidr-assoc-0d6e74c47db6c36c9",
                    "CidrBlock": "172.31.0.0/16",
                    "CidrBlockState": {
                        "State": "associated"
                    }
                }
            ],
            "IsDefault": true
        }
    ]
}


### 4.2 Identify the routing table you want to configure with your VPC Gateway

In [56]:
! aws ec2 describe-route-tables

{
    "RouteTables": [
        {
            "Associations": [
                {
                    "Main": true,
                    "RouteTableAssociationId": "rtbassoc-0ca5602dd6ff0b069",
                    "RouteTableId": "rtb-02be98e72fc4f5ef7",
                    "AssociationState": {
                        "State": "associated"
                    }
                }
            ],
            "PropagatingVgws": [],
            "RouteTableId": "rtb-02be98e72fc4f5ef7",
            "Routes": [
                {
                    "DestinationCidrBlock": "172.31.0.0/16",
                    "GatewayId": "local",
                    "Origin": "CreateRouteTable",
                    "State": "active"
                },
                {
                    "DestinationCidrBlock": "0.0.0.0/0",
                    "GatewayId": "igw-02385e932565c21bf",
                    "Origin": "CreateRoute",
                    "State": "active"
                },
                {
           

### 4.3 Create an S3 Gateway Endpoint

In [15]:
! aws ec2 create-vpc-endpoint --vpc-id 'vpc-0f260bb33946d0513' --service-name com.amazonaws.us-west-2.s3 --route-table-ids 'rtb-02be98e72fc4f5ef7'


An error occurred (RouteAlreadyExists) when calling the CreateVpcEndpoint operation: route table rtb-02be98e72fc4f5ef7 already has a route with destination-prefix-list-id pl-68a54001


### 4.4 Creating the Glue Service IAM Role

In [16]:
! aws iam create-role --role-name stedi-role --assume-role-policy-document '{ "Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Principal": {"Service": "glue.amazonaws.com"}, "Action": "sts:AssumeRole"}]}'


An error occurred (AccessDenied) when calling the CreateRole operation: User: arn:aws:iam::004668495805:user/anhdtv is not authorized to perform: iam:CreateRole on resource: arn:aws:iam::004668495805:role/stedi-role with an explicit deny in an identity-based policy


### 4.5 Grant Glue Privileges on the S3 Bucket

In [17]:
! aws iam put-role-policy --role-name stedi-role --policy-name S3Access --policy-document '{ \
    "Version": "2012-10-17", \
    "Statement": [ \
        { \
            "Effect": "Allow", \
            "Action": [ \
                "s3:GetBucketLocation", \
                "s3:ListBucket", \
                "s3:GetObject", \
                "s3:GetObjectTagging" \
            ], \
            "Resource": [ \
                "arn:aws:s3:::cd0030bucket", \
                "arn:aws:s3:::cd0030bucket/*" \
            ] \
        }, \
        { \
            "Effect": "Allow", \
            "Action": [ \
                "s3:GetBucketLocation", \
                "s3:ListBucket", \
                "s3:GetObject", \
                "s3:PutObject", \
                "s3:deleteObject", \
                "s3:GetObjectTagging", \
                "s3:PutObjectTagging" \
            ], \
            "Resource": [ \
                "arn:aws:s3:::anhdtv-stedi",\
                "arn:aws:s3:::anhdtv-stedi/*" \
            ] \
        } \
    ] \
}'


An error occurred (AccessDenied) when calling the PutRolePolicy operation: User: arn:aws:iam::004668495805:user/anhdtv is not authorized to perform: iam:PutRolePolicy on resource: role stedi-role with an explicit deny in an identity-based policy


### 4.5 Glu Policy - Give Glue access to data in special S3 buckets used for Glue configuration, and several other resources

In [18]:
! aws iam put-role-policy --role-name stedi-role --policy-name GlueAccess --policy-document '{ \
    "Version": "2012-10-17", \
    "Statement": [ \
        { \
            "Effect": "Allow", \
            "Action": [ \
                "glue:*", \
                "s3:GetBucketLocation", \
                "s3:ListBucket", \
                "s3:ListAllMyBuckets", \
                "s3:GetBucketAcl", \
                "ec2:DescribeVpcEndpoints", \
                "ec2:DescribeRouteTables", \
                "ec2:CreateNetworkInterface", \
                "ec2:DeleteNetworkInterface", \
                "ec2:DescribeNetworkInterfaces", \
                "ec2:DescribeSecurityGroups", \
                "ec2:DescribeSubnets", \
                "ec2:DescribeVpcAttribute", \
                "iam:ListRolePolicies", \
                "iam:GetRole", \
                "iam:GetRolePolicy", \
                "cloudwatch:PutMetricData" \
            ], \
            "Resource": [ \
                "*" \
            ] \
        }, \
        { \
            "Effect": "Allow", \
            "Action": [ \
                "s3:CreateBucket", \
                "s3:PutBucketPublicAccessBlock" \
            ], \
            "Resource": [ \
                "arn:aws:s3:::aws-glue-*" \
            ] \
        }, \
        { \
            "Effect": "Allow", \
            "Action": [ \
                "s3:GetObject", \
                "s3:PutObject", \
                "s3:DeleteObject" \
            ], \
            "Resource": [ \
                "arn:aws:s3:::aws-glue-*/*", \
                "arn:aws:s3:::*/*aws-glue-*/*" \
            ] \
        }, \
        { \
            "Effect": "Allow", \
            "Action": [ \
                "s3:GetObject" \
            ], \
            "Resource": [ \
                "arn:aws:s3:::crawler-public*", \
                "arn:aws:s3:::aws-glue-*" \
            ] \
        }, \
        { \
            "Effect": "Allow", \
            "Action": [ \
                "logs:CreateLogGroup", \
                "logs:CreateLogStream", \
                "logs:PutLogEvents", \
                "logs:AssociateKmsKey" \
            ], \
            "Resource": [ \
                "arn:aws:logs:*:*:/aws-glue/*" \
            ] \
        }, \
        { \
            "Effect": "Allow", \
            "Action": [ \
                "ec2:CreateTags", \
                "ec2:DeleteTags" \
            ], \
            "Condition": { \
                "ForAllValues:StringEquals": { \
                    "aws:TagKeys": [ \
                        "aws-glue-service-resource" \
                    ] \
                } \
            }, \
            "Resource": [ \
                "arn:aws:ec2:*:*:network-interface/*", \
                "arn:aws:ec2:*:*:security-group/*", \
                "arn:aws:ec2:*:*:instance/*" \
            ] \
        } \
    ] \
}'


An error occurred (AccessDenied) when calling the PutRolePolicy operation: User: arn:aws:iam::004668495805:user/anhdtv is not authorized to perform: iam:PutRolePolicy on resource: role stedi-role with an explicit deny in an identity-based policy


## 5. Glue

In [78]:
glue = boto3.client('glue')

/datadrive05/anhdtv7/miniconda3/envs/aws/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


### 5.1 Create glue database

In [79]:
glue.create_database(DatabaseInput={'Name': GLUE_DB})

{'ResponseMetadata': {'RequestId': '4e579e7f-6c9d-4a85-b31a-36ace7722a1c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Nov 2023 08:02:30 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4e579e7f-6c9d-4a85-b31a-36ace7722a1c'},
  'RetryAttempts': 0}}

### 5.3 Create Glue tables

In [80]:
# create Glue tables
table_input_customer = {
    'Name': 'customer_landing',
    'Description': 'Customer landing table from S3',
    'StorageDescriptor': {
        'Columns': [
            {'Name': 'customerName', 'Type': 'string'},
            {'Name': 'email', 'Type': 'string'},
            {'Name': 'phone', 'Type': 'string'},
            {'Name': 'birthDay', 'Type': 'string'},
            {'Name': 'serialNumber', 'Type': 'string'},
            {'Name': 'registrationDate', 'Type': 'bigint'},
            {'Name': 'lastUpdateDate', 'Type': 'bigint'},
            {'Name': 'shareWithResearchAsOfDate', 'Type': 'bigint'},
            {'Name': 'shareWithPublicAsOfDate', 'Type': 'bigint'},
        ],
        'Location': 's3://anhdtv-stedi/customer/landing/',
        'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
        'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
        'SerdeInfo': {
            'SerializationLibrary': 'org.openx.data.jsonserde.JsonSerDe',
            'Parameters': { 'separatorChar': ',' }
        },
        'StoredAsSubDirectories': False
    },
    'TableType': 'EXTERNAL_TABLE'
}

table_input_accelerometer = {
    'Name': 'accelerometer_landing',
    'Description': 'Accelerometer landing table from S3',
    'StorageDescriptor': {
        'Columns': [
            {'Name': 'user', 'Type': 'string'},
            {'Name': 'timeStamp', 'Type': 'bigint'},
            {'Name': 'x', 'Type': 'double'},
            {'Name': 'y', 'Type': 'double'},
            {'Name': 'z', 'Type': 'double'},
        ],
        'Location': 's3://anhdtv-stedi/accelerometer/landing/',
        'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
        'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
        'SerdeInfo': {
            'SerializationLibrary': 'org.openx.data.jsonserde.JsonSerDe',
            'Parameters': {'separatorChar': ','}
        },
        'StoredAsSubDirectories': False
    },
    'TableType': 'EXTERNAL_TABLE'
}

glue.create_table(DatabaseName=GLUE_DB, TableInput=table_input_customer)
glue.create_table(DatabaseName=GLUE_DB, TableInput=table_input_accelerometer)

{'ResponseMetadata': {'RequestId': 'edeb8993-2f5c-4b7b-8185-3b5a0c9aebfe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Nov 2023 08:02:42 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'edeb8993-2f5c-4b7b-8185-3b5a0c9aebfe'},
  'RetryAttempts': 0}}

### 5.4 Create customer trusted table

In [81]:
table_input_customer_trusted = {
    'Name': 'customer_trusted',
    'Description': 'Customer trusted table from S3',
    'StorageDescriptor': {
        'Columns': [
            {'Name': 'customerName', 'Type': 'string'},
            {'Name': 'email', 'Type': 'string'},
            {'Name': 'phone', 'Type': 'string'},
            {'Name': 'birthDay', 'Type': 'string'},
            {'Name': 'serialNumber', 'Type': 'string'},
            {'Name': 'registrationDate', 'Type': 'bigint'},
            {'Name': 'lastUpdateDate', 'Type': 'bigint'},
            {'Name': 'shareWithResearchAsOfDate', 'Type': 'bigint'},
            {'Name': 'shareWithPublicAsOfDate', 'Type': 'bigint'},
        ],
        'Location': 's3://anhdtv-stedi/customer/trusted/',
        'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
        'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
        'SerdeInfo': {
            'SerializationLibrary': 'org.openx.data.jsonserde.JsonSerDe',
            'Parameters': { 'separatorChar': ',' }
        },
        'StoredAsSubDirectories': False
    },
    'TableType': 'EXTERNAL_TABLE'
}

glue.create_table(DatabaseName=GLUE_DB, TableInput=table_input_customer_trusted)

{'ResponseMetadata': {'RequestId': '9453c94e-1964-4d6c-a3bf-178858df03db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Nov 2023 08:02:45 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9453c94e-1964-4d6c-a3bf-178858df03db'},
  'RetryAttempts': 0}}

### 5.4 Create Accelerometer trusted table

In [61]:
table_input_accelerometer_trusted = {
    'Name': 'accelerometer_trusted',
    'Description': 'Accelerometer trusted table from S3',
    'StorageDescriptor': {
        'Columns': [
            {'Name': 'user', 'Type': 'string'},
            {'Name': 'timeStamp', 'Type': 'bigint'},
            {'Name': 'x', 'Type': 'double'},
            {'Name': 'y', 'Type': 'double'},
            {'Name': 'z', 'Type': 'double'},
        ],
        'Location': 's3://anhdtv-stedi/accelerometer/trusted/',
        'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
        'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
        'SerdeInfo': {
            'SerializationLibrary': 'org.openx.data.jsonserde.JsonSerDe',
            'Parameters': {'separatorChar': ','}
        },
        'StoredAsSubDirectories': False
    },
    'TableType': 'EXTERNAL_TABLE'
}

glue.create_table(DatabaseName=GLUE_DB, TableInput=table_input_accelerometer_trusted)

{'ResponseMetadata': {'RequestId': 'ba6dd7c8-8b64-4f15-8bf4-5aac2b16f97a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Nov 2023 07:57:33 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ba6dd7c8-8b64-4f15-8bf4-5aac2b16f97a'},
  'RetryAttempts': 0}}

### 5.5 Create Customer Curated table

In [82]:
table_input_customer_curated = {
    'Name': 'customer_curated',
    'Description': 'Customer Curated table from S3',
    'StorageDescriptor': {
        'Columns': [
            {'Name': 'customerName', 'Type': 'string'},
            {'Name': 'email', 'Type': 'string'},
            {'Name': 'phone', 'Type': 'string'},
            {'Name': 'birthDay', 'Type': 'string'},
            {'Name': 'serialNumber', 'Type': 'string'},
            {'Name': 'registrationDate', 'Type': 'bigint'},
            {'Name': 'lastUpdateDate', 'Type': 'bigint'},
            {'Name': 'shareWithResearchAsOfDate', 'Type': 'bigint'},
            {'Name': 'shareWithPublicAsOfDate', 'Type': 'bigint'},
        ],
        'Location': 's3://anhdtv-stedi/customer/curated/',
        'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
        'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
        'SerdeInfo': {
            'SerializationLibrary': 'org.openx.data.jsonserde.JsonSerDe',
            'Parameters': { 'separatorChar': ',' }
        },
        'StoredAsSubDirectories': False
    },
    'TableType': 'EXTERNAL_TABLE'
}

glue.create_table(DatabaseName=GLUE_DB, TableInput=table_input_customer_curated)

{'ResponseMetadata': {'RequestId': '011eb944-bc47-469a-b249-2d3026878a7a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Nov 2023 08:02:51 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '011eb944-bc47-469a-b249-2d3026878a7a'},
  'RetryAttempts': 0}}

### 5.5 Create Step Trainer trusted table

In [63]:
table_input_step_trainer_trusted = {
    'Name': 'step_trainer_trusted',
    'Description': 'Step Trainer trusted table from S3',
    'StorageDescriptor': {
        'Columns': [
            {'Name': 'sensorReadingTime', 'Type': 'bigint'},
            {'Name': 'serialNumber', 'Type': 'string'},
            {'Name': 'distanceFromObject', 'Type': 'int'},
        ],
        'Location': 's3://anhdtv-stedi/step_trainer/trusted/',
        'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
        'OutputFormat': 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat',
        'SerdeInfo': {
            'SerializationLibrary': 'org.openx.data.jsonserde.JsonSerDe',
            'Parameters': {'separatorChar': ','}
        },
        'StoredAsSubDirectories': False
    },
    'TableType': 'EXTERNAL_TABLE'
}

glue.create_table(DatabaseName=GLUE_DB, TableInput=table_input_step_trainer_trusted)

{'ResponseMetadata': {'RequestId': '1fa769da-ab04-4f25-89ba-14b389b1689b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Nov 2023 07:57:39 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1fa769da-ab04-4f25-89ba-14b389b1689b'},
  'RetryAttempts': 0}}

## 6 - AWS Athena

### 6.1 Create Athena client object

In [83]:
athena = boto3.client('athena')

/datadrive05/anhdtv7/miniconda3/envs/aws/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [84]:

response = athena.start_query_execution(
    QueryString='CREATE DATABASE stedi',
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")

Execution ID: d185da42-5465-46e6-b16e-c37445993c64


In [66]:

response = athena.start_query_execution(
    QueryString='DROP DATABASE IF EXISTS stedi',
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")

Execution ID: d15506b2-1f98-4624-9392-d36e89477bc4


In [29]:
# Read the SQL script file
with open('./sql_script/accelerometer_landing.sql', 'r') as f:
    query_string = f.read()

response = athena.start_query_execution(
    QueryString=query_string,
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/accelerometer/landing'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")


Execution ID: 92bdba62-3812-4497-8820-317f2ad0ee7d


In [67]:
# count rows of accelerometer_landing
response = athena.start_query_execution(
    QueryString="""SELECT
                    COUNT(*) AS number_of_rows
                FROM stedi.accelerometer_landing""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/accelerometer/landing'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")


Execution ID: 5ae68db9-063a-4f92-9238-ccf8786296ef


In [42]:
# count rows of accelerometer_landing
response = athena.start_query_execution(
    QueryString="""SELECT COUNT(*) 
                FROM stedi.accelerometer_landing""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/accelerometer/landing'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")


Execution ID: 48d244e4-c642-4973-bc45-4fa8974e3939


In [90]:
# count rows of accelerometer_landing
response = athena.start_query_execution(
    QueryString="""SELECT COUNT(*) 
                FROM stedi.accelerometer_trusted""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/accelerometer/trusted'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")


Execution ID: 0922e60f-cfe3-41fe-9526-39becc8ba77f


In [85]:
# Read the SQL script file
with open('./sql_script/customer_landing.sql', 'r') as f:
    query_string = f.read()

response = athena.start_query_execution(
    QueryString=query_string,
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/customer/landing'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")


Execution ID: 004a2f0c-7524-4acd-aed0-9540246da39a


In [86]:
# count rows of customer_landing
response = athena.start_query_execution(
    QueryString="""SELECT
                    COUNT(*) AS number_of_rows
                FROM stedi.customer_landing""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/customer/landing'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")

Execution ID: e73b15ec-081a-4842-a177-04bea63084b0


In [89]:
# count rows of customer_landing
response = athena.start_query_execution(
    QueryString="""SELECT
                    COUNT(*) AS number_of_rows
                FROM stedi.customer_trusted""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/customer/trusted'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")

Execution ID: a72c6da3-4404-4eaa-8205-5bf9ec6aeb7d


In [32]:
# count rows of customer_landing
response = athena.start_query_execution(
    QueryString="""SELECT *
                FROM stedi.customer_trusted""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/customer/trusted'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")

Execution ID: 10698127-6b91-4871-b7c6-b06694775c0f


In [33]:
response = athena.start_query_execution(
    QueryString='DROP TABLE IF EXISTS stedi.customer_landing',
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/customer/landing'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")


Execution ID: 51f30f1a-e7b8-497a-bdf1-9a995233c1f7


In [34]:
# count rows of customer_landing
response = athena.start_query_execution(
    QueryString="""SELECT *
                FROM stedi.customer_curated""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/customer/curated'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")


Execution ID: 81d3c194-e5cf-4c62-8450-843662d510bf


In [39]:
# count rows of customer_landing
response = athena.start_query_execution(
    QueryString="""SELECT COUNT(*) AS number_of_rows
                FROM stedi.customer_landing""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/customer/landing'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")


Execution ID: 64b5d750-c58c-461f-b7de-4e74237cf866


In [35]:
# Read the SQL script file
with open('./sql_script/step_trainer_landing.sql', 'r') as f:
    query_string = f.read()

response = athena.start_query_execution(
    QueryString=query_string,
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/step_trainer/landing'
    }
)

execution_id = response['QueryExecutionId']
print(f"Execution ID: {execution_id}")


Execution ID: 15496593-ba06-4e94-8107-9d357e566c0f


In [36]:
# count rows of accelerometer_landing
response = athena.start_query_execution(
    QueryString="""SELECT
                    COUNT(*) AS number_of_rows
                FROM stedi.step_trainer_landing""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/step_trainer/landing'
    }
)

In [37]:
# count rows of accelerometer_landing
response = athena.start_query_execution(
    QueryString="""SELECT
                    *
                FROM stedi.step_trainer_trusted""",
    QueryExecutionContext={
        'Database': 'stedi'
    },
    ResultConfiguration={
        'OutputLocation': 's3://anhdtv-stedi/step_trainer/trusted'
    }
)

## 7. Delete Resource

### Delete S3 bucket 

In [91]:
try:
        bucket = s3.Bucket(S3_BUCKET_NAME)
        bucket.objects.all().delete()

        # Delete the bucket
        bucket.delete()
        print(f'Successfully deleted S3 bucket: {S3_BUCKET_NAME}')
except ClientError as e:
    if e.response['Error']['Code'] == 'BucketNotEmpty':
        print(f'S3 bucket {S3_BUCKET_NAME} is not empty. Deleting all objects inside the bucket...')
        
        s3.objects.all().delete()
        s3.delete_bucket(Bucket=S3_BUCKET_NAME)
        print(f'Successfully deleted S3 bucket: {S3_BUCKET_NAME}')
    elif e.response['Error']['Code'] == 'NoSuchBucket':
        print(f'S3 bucket {S3_BUCKET_NAME} does not exist')
    else:
        print(f'Error deleting S3 bucket {S3_BUCKET_NAME}: {e}')

# Wait for the bucket to be deleted
while True:
    try:
        s3.meta.client.head_bucket(Bucket=S3_BUCKET_NAME)
        time.sleep(10)
    except ClientError as e:
        if e.response['Error']['Code'] == '404':
            print(f'S3 bucket {S3_BUCKET_NAME} successfully deleted')
            break
        else:
            print(f'Error waiting for S3 bucket {S3_BUCKET_NAME} to be deleted: {e}')


Error deleting S3 bucket anhdtv-stedi: An error occurred (AccessDenied) when calling the DeleteBucket operation: Access Denied


NameError: name 'time' is not defined

### Delete Glue DB

In [92]:
glue.delete_database(Name=GLUE_DB)

{'ResponseMetadata': {'RequestId': 'e6c799f3-3fb5-49e1-9996-26b6c45b7876',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 06 Nov 2023 08:41:35 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e6c799f3-3fb5-49e1-9996-26b6c45b7876'},
  'RetryAttempts': 0}}